In [1]:

import pandas as pd
import numpy as np
import pyodbc
import openpyxl as op
import os
import time
import matplotlib.pyplot as plt
import datetime

pd.set_option("display.max_columns", 50) #최대 50개의 컬럼까지 줄바꿈으로 보여줌
file_path = "T:\\index\\95_곽용하\\연구\\0_CA\\무상증자\\"
conn_wisefn = pyodbc.connect('driver={SQL Server};server=46.2.90.172;database=wisefn;uid=index;pwd=samsung@00')

In [2]:

# data
# 1) 무상증자 관련 데이터
data1 = pd.read_excel(file_path + "BT_무증_RAWDATA.xlsx", sheet_name='(3)_Adj', header = 0, dtype={'공시일_str':str,'권리락일_str':str,'상장일_str':str})
stk_list = tuple([x[1:] for x in list(data1['기업코드'].unique())])

# 2) 일자 데이터
sql_date = '''  SELECT YMD AS TRD_DT, CONVERT(CHAR(8),TRDT_1,112) 'B4_1'
                FROM WISEFN..TZ_DATE
                WHERE TRADE_YN = 1
                AND YMD BETWEEN '20170601' AND '20250301'  '''
date2 = pd.read_sql(sql_date, conn_wisefn)
date2_list = list(date2['TRD_DT'])
# 3) 수정주가 데이터
data3 = pd.read_excel(file_path + "BT_무증_RAWDATA.xlsx", sheet_name='adj_prc', skiprows=7)
data3 = data3.iloc[6:,:]
data3['Code'] = pd.to_datetime(data3['Code'], format='%Y%m%d')
data3['Code'] = data3['Code'].dt.strftime('%Y%m%d')
data3 = data3.rename(columns={'Code':'TRD_DT'})
data3.set_index('TRD_DT',inplace=True)
data3 = data3.astype(float)
data3_log = np.log(data3)

In [3]:
data3_log

,A128940,A102940,A251370,A000100,A001060,A096760,A008930,A014100,A182400,A035200,A079000,A002240,A017180,A214870,A900280,A185750,A018620,A140070,A122640,A187790,A003220,A230360,A219420,A101330,A187420,...,A347770,A114450,A241820,A298060,A223250,A205100,A024720,A270520,A214610,A214320,A177350,A115180,A051380,A290690,A053980,A106190,A336370,A079370,A192080,A314930,A357230,A089850,A340930,A042500,A100840
TRD_DT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20161229,12.431502,11.268597,NaN,10.287627,10.700950,8.920923,10.843827,9.592673,9.093807,8.776476,8.952864,10.273533,8.500657,8.109225,7.804659,11.228531,8.606302,NaN,9.404920,8.362409,9.676336,8.614501,7.251345,8.651374,9.182455,...,NaN,8.179200,NaN,NaN,NaN,7.479864,9.732818,NaN,9.264734,10.259412,7.570959,10.103567,7.659171,NaN,8.975377,9.204322,NaN,8.419139,10.426647,NaN,NaN,8.593228,NaN,8.148446,8.793915
20170102,12.360286,11.249676,NaN,10.285104,10.687572,8.901639,10.769095,9.579418,9.068201,8.776476,8.947936,10.277256,8.491260,8.128585,7.793999,11.194468,8.603554,NaN,9.394993,8.325306,9.655731,8.612140,7.240650,8.651374,9.182455,...,NaN,8.179200,NaN,NaN,NaN,7.483807,9.613202,NaN,9.103868,10.254144,7.592366,10.127471,7.625595,NaN,8.959954,9.183996,NaN,8.486734,10.392160,NaN,NaN,8.593228,NaN,8.157657,8.785998
20170103,12.369033,11.246196,NaN,10.280073,10.706632,8.908559,10.779623,9.589393,9.079662,8.794219,8.938138,10.266045,8.478036,8.128585,7.821643,11.180511,8.581107,NaN,9.394993,8.367532,9.634693,8.567886,7.085064,8.655911,9.177301,...,NaN,8.195885,NaN,NaN,NaN,7.548556,9.577411,NaN,9.055089,10.259412,7.629976,10.130264,7.613325,NaN,8.978913,9.183996,NaN,8.493515,10.407621,NaN,NaN,8.593228,NaN,8.182839,8.801620
20170104,12.401598,11.289657,NaN,10.285104,10.727158,8.920923,10.808859,9.615539,9.106423,8.794219,8.939319,10.272289,8.500657,8.124151,7.828835,11.194468,8.583917,NaN,9.378225,8.380457,9.632007,8.563122,7.223296,8.663715,9.210340,...,NaN,8.215547,NaN,NaN,NaN,7.556428,9.559940,NaN,9.099856,10.255903,7.613325,10.176335,7.618251,NaN,8.978913,9.200997,NaN,8.517193,10.388257,NaN,NaN,8.598036,NaN,8.175829,8.801620
20170105,12.404964,11.311115,NaN,10.314835,10.719737,8.908559,10.813921,9.615539,9.106423,8.791638,8.943114,10.279764,8.502486,8.101678,7.832014,11.228531,8.588211,NaN,9.357639,8.382976,9.671240,8.541300,7.363280,8.702178,9.230143,...,NaN,8.209308,NaN,NaN,NaN,7.570959,9.558176,NaN,9.095827,10.292146,7.605890,10.167043,7.613325,NaN,8.981304,9.197559,NaN,8.627482,10.379100,NaN,NaN,8.583917,NaN,8.171317,8.806873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20240412,12.663497,10.010097,9.402612,11.173248,10.327120,8.006368,10.463103,8.072467,7.610853,8.086410,8.517193,9.918376,7.656810,7.232010,4.644391,11.540541,7.029973,8.184514,9.910960,6.932448,9.560997,9.586720,9.074521,8.103192,8.268732,...,8.255828,7.573017,7.148346,7.930206,7.969012,7.757906,8.982184,7.851661,7.313220,9.971146,6.052089,8.425078,8.198364,7.932003,8.468003,9.183996,9.898977,9.745078,10.597884,9.079092,8.321178,9.136694,8.526747,8.791334,9.404343
20240415,12.631340,10.007848,9.369905,11.153389,10.305614,7.989560,10.431170,8.061487,7.610853,8.086410,8.507143,9.910960,7.635304,7.232010,4.624973,11.508917,7.008505,8.167636,9.975808,6.932448,9.560997,9.540363,9.016756,8.100161,8.258422,...,8.233503,7.534228,7.140453,7.882315,7.944492,7.745003,8.985946,7.843849,7.303170,9.971146,6.091310,8.420682,8.216088,7.932003,8.466952,9.162200,9.786954,9.736842,10.585321,9.060680,8.290293,9.129130,8.571681,8.776476,9.396820
20240416,12.611538,9.989665,9.323669,11.146200,10.268131,7.969012,10.398184,8.086410,7.610853,8.053569,8.508152,9.908475,7.630461,7.232010,4.584967,11.513925,6.998510,8.150468,9.930616,6.932448,9.546098,9.556763,8.952864,8.078688,8.202482,...,8.201112,7.560601,7.130899,7.874739,7.886081,7.705262,8.945072,7.781139,7.313220,

In [4]:
df1 = data1.copy()
df1['공시_권리락'] = df1['기업코드']
df1['권리락_상장'] = df1['기업코드']
for i in range(len(df1)):
    df1['공시_권리락'][i] = len([x for x in date2_list if (x >= data1['공시일_str'][i])   & (x <= data1['권리락일_str'][i])])
    df1['권리락_상장'][i] = len([x for x in date2_list if (x >= data1['권리락일_str'][i]) & (x <= data1['상장일_str'][i])])
df1 = df1[df1['공시_권리락']>0]
df1 = df1.reset_index(drop=True)

<ipython-input-4-bef9e09a57cd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['공시_권리락'][i] = len([x for x in date2_list if (x >= data1['공시일_str'][i])   & (x <= data1['권리락일_str'][i])])
<ipython-input-4-bef9e09a57cd>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['권리락_상장'][i] = len([x for x in date2_list if (x >= data1['권리락일_str'][i]) & (x <= data1['상장일_str'][i])])


In [5]:
df1

,기업코드,기업명,공시일,권리락일,상장일,배정비율,변동주식수,총발행주식수,증자비율,공시일_str,권리락일_str,상장일_str,공시_권리락,권리락_상장
0,A128940,한미약품,2017-12-14,2017-12-27,2018-01-23,0.02,222239,11385691,0.019908,20171214,20171227,20180123,9,18
1,A102940,코오롱생명과학,2017-12-08,2017-12-27,2018-01-25,0.50,3801466,11412356,0.499477,20171208,20171227,20180125,13,20
2,A251370,와이엠티,2017-12-11,2017-12-27,2018-01-19,1.00,3702600,7405200,1.000000,20171211,20171227,20180119,12,16
3,A000100,유한양행,2017-12-13,2017-12-27,2018-01-22,0.05,531975,12197871,0.045601,20171213,20171227,20180122,10,17
4,A001060,JW중외제약,2017-12-15,2017-12-27,2018-01-22,0.02,382685,19533922,0.019982,20171215,20171227,20180122,8,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,A089850,유비벨록스,2024-03-19,2024-04-02,2024-05-03,1.00,7273465,14730199,0.975422,20240319,20240402,20240503,11,12
492,A315640,딥노이드,2024-03-25,2024-04-08,2024-05-03,1.00,11210634,22421268,1.000000,20240325,20240408,20240503,11,8
493,A340930,유일에너테크,2024-04-01,2024-04-16,2024-05-07,2.00,22736300,34204450,1.982560,20240401,20240416,20240507,11,3
494,A042500,링네트,2024-04-01,2024-04-16,2024-05-13,0.30,3713256,19070134,0.241798,20240401,20240416,20240513,11,3


In [6]:
# 구간별 최대일자 구하기
ntfi_lck_max = max(df1['공시_권리락']) # 최대값: 공시~권리락
lck_stt_max  = max(df1['권리락_상장']) # 최대값: 권리락~상장일

In [7]:
print(ntfi_lck_max)

258


In [8]:
# 주가 데이터 수정
prc_stack = data3_log.stack().reset_index()
prc_stack.columns = ['TRD_DT','기업코드','log_prc']
prc_stack

,TRD_DT,기업코드,log_prc
0,20161229,A128940,12.431502
1,20161229,A102940,11.268597
2,20161229,A000100,10.287627
3,20161229,A001060,10.700950
4,20161229,A096760,8.920923
...,...,...,...
601417,20240418,A357230,8.276395
601418,20240418,A089850,9.100526
601419,20240418,A340930,8.538955
601420,20240418,A042500,8.853665
